In [ ]:
import torch
import pickle
import numpy as np
import random
from scipy import spatial

In [ ]:
emb = torch.load('./state/fb15k237_fed3_TransE.best')

In [ ]:
ent_embed = emb['ent_embed']

In [ ]:
rel_embed = emb['rel_embed']

#### ERR

In [ ]:
data = pickle.load(open("FB15K237-Fed3.pkl", "rb" ))

In [ ]:
c0_ent = np.unique(data[0]['train']['edge_index_ori'])
c1_ent = np.unique(data[1]['train']['edge_index_ori'])
c2_ent = np.unique(data[2]['train']['edge_index_ori'])

In [ ]:
print(len(c0_ent), len(c1_ent), len(c2_ent))

In [ ]:
random.seed(10)
np.random.seed(10)
n = int(1 * len(c0_ent))
pool = np.random.choice(c0_ent, n, replace = False)
print(len(np.intersect1d(pool, c1_ent))/len(c1_ent))
print(len(np.intersect1d(pool, c2_ent))/len(c2_ent))
#pool = pool.tolist()

#### TRR

In [ ]:
# creat entity pool of selected entities (adversary knowledge)
c0_ent_embed_dict = {}
value = ent_embed[[pool]]
for idx,ent in enumerate(pool):
    c0_ent_embed_dict[ent] = value[idx]

In [ ]:
# creat relation pool based on selected entities
c0_triple_all = np.array([data[0]['train']['edge_index_ori'][0], 
                          data[0]['train']['edge_type_ori'], 
                          data[0]['train']['edge_index_ori'][1]])

len_c0_triple_all = c0_triple_all[0].shape[0]
rel_pool = []
for i in range(len_c0_triple_all):
    triple = c0_triple_all[:,i]
    if (triple[0] in pool) or (triple[2] in pool):
        if triple[1] not in rel_pool:
            rel_pool.append(triple[1])

In [ ]:
# create relation embedding dictionary
c0_rel_embed_dict = {}
value = rel_embed[0][[rel_pool]]
for idx,rel in enumerate(rel_pool):
    c0_rel_embed_dict[idx] = value[idx]

In [ ]:
# client 0 synthetic relation list
syn_rel_list = [] # synthetic relation label

c0_ent_all = data[0]['train']['edge_index_ori'] # entities in client 0

for i in range(len(c0_ent_all[0])):
    if (c0_ent_all[0][i] in pool) and (c0_ent_all[1][i] in pool):
        syn_rel_embed = c0_ent_embed_dict[c0_ent_all[1][i]] - c0_ent_embed_dict[c0_ent_all[0][i]]
        if torch.sum(syn_rel_embed) == 0:
            syn_rel_embed += 1e-12
        cos_loss = []
        for j in c0_rel_embed_dict:
            loss = spatial.distance.cosine(syn_rel_embed.cpu().detach().numpy(), c0_rel_embed_dict[j].cpu().detach().numpy())
            cos_loss.append(loss)
        rel_idx_in_lst = cos_loss.index(min(cos_loss))
        rel_idx = rel_pool[rel_idx_in_lst]
        syn_rel_list.append(rel_idx)

In [ ]:
tru_rel_list = []
for i in range(len_c0_triple_all):
    triple = c0_triple_all[:,i]
    if (triple[0] in pool) and (triple[2] in pool):
        tru_rel_list.append(triple[1])

In [ ]:
# calculate the number of correct reconstruction
sum(first == second for (first, second) in zip(syn_rel_list, tru_rel_list)) / len_c0_triple_all

In [ ]:
# client 1 synthetic relation list
syn_rel_list = [] # synthetic relation label
# ent_list = [] # store entity index for calculate the reconstruction

c1_ent_all = data[1]['train']['edge_index_ori'] # entities in client 1

for i in range(len(c1_ent_all[0])):
    if (c1_ent_all[0][i] in pool) and (c1_ent_all[1][i] in pool):
        syn_rel_embed = c0_ent_embed_dict[c1_ent_all[1][i]] - c0_ent_embed_dict[c1_ent_all[0][i]]
        cos_loss = []
        for j in c0_rel_embed_dict:
            loss = spatial.distance.cosine(syn_rel_embed.cpu().detach().numpy(), c0_rel_embed_dict[j].cpu().detach().numpy())
            cos_loss.append(loss)
        rel_idx_in_lst = cos_loss.index(min(cos_loss))
        rel_idx = rel_pool[rel_idx_in_lst]
        syn_rel_list.append(rel_idx)

In [ ]:
# client 1 ground-truth relation list
c1_triple_all = np.array([data[1]['train']['edge_index_ori'][0], 
                          data[1]['train']['edge_type_ori'], 
                          data[1]['train']['edge_index_ori'][1]])

len_c1_triple_all = c1_triple_all[0].shape[0]
tru_rel_list = []
for i in range(len_c1_triple_all):
    triple = c1_triple_all[:,i]
    if (triple[0] in pool) and (triple[2] in pool):
        tru_rel_list.append(triple[1])

In [ ]:
# calculate the number of correct reconstruction
sum(first == second for (first, second) in zip(syn_rel_list, tru_rel_list)) / len_c1_triple_all

In [ ]:
# client 2 synthetic relation list
syn_rel_list = [] # synthetic relation label
# ent_list = [] # store entity index for calculate the reconstruction

c2_ent_all = data[2]['train']['edge_index_ori'] # entities in client 2

for i in range(len(c2_ent_all[0])):
    if (c2_ent_all[0][i] in pool) and (c2_ent_all[1][i] in pool):
        syn_rel_embed = c0_ent_embed_dict[c2_ent_all[1][i]] - c0_ent_embed_dict[c2_ent_all[0][i]]
        cos_loss = []
        for j in c0_rel_embed_dict:
            loss = spatial.distance.cosine(syn_rel_embed.cpu().detach().numpy(), c0_rel_embed_dict[j].cpu().detach().numpy())
            cos_loss.append(loss)
        rel_idx_in_lst = cos_loss.index(min(cos_loss))
        rel_idx = rel_pool[rel_idx_in_lst]
        syn_rel_list.append(rel_idx)

In [ ]:
# client 2 ground-truth relation list
c2_triple_all = np.array([data[2]['train']['edge_index_ori'][0], 
                          data[2]['train']['edge_type_ori'], 
                          data[2]['train']['edge_index_ori'][1]])

len_c2_triple_all = c2_triple_all[0].shape[0]
tru_rel_list = []
for i in range(len_c2_triple_all):
    triple = c2_triple_all[:,i]
    if (triple[0] in pool) and (triple[2] in pool):
        tru_rel_list.append(triple[1])

In [ ]:
# calculate the number of correct reconstruction
sum(first == second for (first, second) in zip(syn_rel_list, tru_rel_list)) / len_c2_triple_all